In [ ]:
<div class="alert alert-block alert-info"> <b>Transformer Model</b> </div>

<div class="alert alert-block alert-info"> <b>Dependencies</b> </div>


In [2]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential, Model
#from tensorflow.python.keras.layers.convolutional import Conv1D, MaxPooling1D
#from tensorflow.python.keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten
from keras.layers import TimeDistributed
from tensorflow.keras.layers import Dense, Conv1D , MaxPooling1D , LSTM, TimeDistributed, Flatten, GRU, RepeatVector
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


import plotly.offline as py #visualization
py.init_notebook_mode(connected=True) #visualization
import plotly.graph_objs as go #visualization
import plotly.tools as tls #visualization
import plotly.figure_factory as ff


from plotly.offline import init_notebook_mode, iplot

%matplotlib inline
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)

# Set seeds to make the experiment more reproducible.
from tensorflow.compat.v1 import set_random_seed
from numpy.random import seed
set_random_seed(1)
seed(1)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt



import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
import glob

from RNN import RNN
#from CNN import CNN
from Transformer import Transformer

from utils import series_to_supervised


In [14]:
pd.set_option('display.max_columns', None)

In [15]:
train = pd.read_csv('/Users/moumi/Documents/SCU/Python/jupyter/DL Project/demand-forecasting-kernels-only/train.csv', parse_dates=['date'])

In [16]:
train.shape

(913000, 4)

In [17]:
#Time period of the train dataset
print('Min date from train set: %s' % train['date'].min().date())
print('Max date from train set: %s' % train['date'].max().date())

Min date from train set: 2013-01-01
Max date from train set: 2017-12-31


<div class="alert alert-block alert-info"> <b> Data Cleaning and Pre-processing</b> </div>


In [18]:
#adding features to the training dataset to granular level
train['date']=pd.to_datetime(train['date'], infer_datetime_format=True)
train['year'] = train['date'].dt.year
train['month'] = train['date'].dt.month
train['day'] = train['date'].dt.day
train['week'] = train['date'].dt.week
train['dayofweek'] = train['date'].dt.dayofweek
train['weekday'] = train['date'].dt.weekday
train['dayofyear'] = train['date'].dt.dayofyear
train['quarter'] = train['date'].dt.quarter
train['daily_avg']=train.groupby(['item','store','dayofweek'])['sales'].transform('mean')
train['monthly_avg']=train.groupby(['item','store','month'])['sales'].transform('mean')
train["mean_store_item_month"] = train.groupby(['month',"item","store"])["sales"].transform("mean")
daily_avg=train.groupby(['item','store','dayofweek'])['sales'].mean().reset_index()
monthly_avg=train.groupby(['item','store','month'])['sales'].mean().reset_index()
mean_store_item_month = train.groupby(['month','item','store'])['sales'].mean().reset_index()
train.dtypes


date                     datetime64[ns]
store                             int64
item                              int64
sales                             int64
year                              int64
month                             int64
day                               int64
week                              int64
dayofweek                         int64
weekday                           int64
dayofyear                         int64
quarter                           int64
daily_avg                       float64
monthly_avg                     float64
mean_store_item_month           float64
dtype: object

In [19]:
#making a copy of the training data to use to for further processing
train_transformer = train.copy()
train.head()

,date,store,item,sales,year,month,day,week,dayofweek,weekday,dayofyear,quarter,daily_avg,monthly_avg,mean_store_item_month
0,2013-01-01,1,1,13,2013,1,1,1,1,1,1,1,18.168582,13.709677,13.709677
1,2013-01-02,1,1,11,2013,1,2,1,2,2,2,1,18.793103,13.709677,13.709677
2,2013-01-03,1,1,14,2013,1,3,1,3,3,3,1,19.452107,13.709677,13.709677
3,2013-01-04,1,1,13,2013,1,4,1,4,4,4,1,21.015326,13.709677,13.709677
4,2013-01-05,1,1,10,2013,1,5,1,5,5,5,1,22.973180,13.709677,13.709677


<div class="alert alert-block alert-info"> <b>Train, Validation and Test Data Preparation</b> </div>

In [20]:
#creating test dataset from training dataset for deep learning models
test_transformer = train_transformer[(train_transformer['date'] > '2017-08-01')]
train_transformer = train_transformer[(train_transformer['date'] <= '2017-07-31')]
#Dropping target column - date column as other custom created date features will be used instead to train the model
train_transformer= train_transformer.drop(['date'], axis = 1)

In [21]:
train_transformer.shape

(836500, 14)

In [118]:
#making a copy of train and test dataset for machine learning models
#test_copy = test.copy()
#train_copy = train.copy()
#test configurations for machine learning models
x_test_transformer = test_transformer.drop(['sales', 'date'], axis=1)
y_test_transformer = test_transformer.pop('sales')
y_date = test_transformer.pop('date')
#training configurations for machine learning models and splitting validation data set by 20%
x_train,x_val,y_train,y_val = train_test_split(train_copy.drop('sales',axis=1),train_copy.pop('sales'),random_state=42,test_size=0.2)
print('Observations: %d' % (len(train_copy)))
print('Training Observations: %d' % (len(x_train)))
print('Validation Observations: %d' % (len(x_val)))
print('Testing Observations: %d' % (len(x_test)))


Observations: 836500
Training Observations: 669200
Validation Observations: 167300
Testing Observations: 76000


In [119]:
# #making a copy of train and test dataset for deep learning models
# test_v2_copy = test_v2.copy()
# train_v2_copy = train_v2.copy()
# #test configurations for deep learning models
# x_test_v2 = test_v2_copy.drop(['sales', 'date'], axis=1)
# y_test_v2 = test_v2_copy.pop('sales')
# y_date_v2 = test_v2_copy.pop('date')
# #training configurations for deep learning models and splitting validation data set by 20%
# x_train_v2,x_val_v2,y_train_v2,y_val_v2 = train_test_split(train_v2_copy.drop('sales',axis=1),train_v2_copy.pop('sales'),random_state=42,test_size=0.2)
# print('Observations: %d' % (len(train_v2_copy)))
# print('Training Observations: %d' % (len(x_train_v2)))
# print('Validation Observations: %d' % (len(x_val_v2)))
# print('Testing Observations: %d' % (len(x_test_v2)))


Observations: 836500
Training Observations: 669200
Validation Observations: 167300
Testing Observations: 76000


In [25]:
#introducing look-back period of 14 days
look_back = 14
n_features = train_transformer.shape[1]
n_features

14

In [26]:
# Walk-forward data split to avoid data leakage
X_train, y_train, X_test, y_test, scale_X = series_to_supervised(train_transformer, train_size=0.8, n_in=look_back, n_out=14, target_column='sales', dropnan=True, scale_X=True)


In [ ]:
# reshape input to be 3D [samples, timesteps, features]
X_train_reshaped = X_train.values.reshape((-1,look_back,n_features))
X_test_reshaped = X_test.values.reshape((-1,look_back,n_features))

y_train_reshaped = y_train.values
y_test_reshaped = y_test.values

tr = Transformer()
tr.train(X_train_reshaped,y_train_reshaped)
_, rmse_result, mae_result, smape_result, r2_result = tr.evaluate(X_test_reshaped,y_test_reshaped)


print('Result \n RMSE = %.2f [kWh] \n MAE = %.2f [kWh]\n R2 = %.1f [%%]' % (rmse_result,
                                                                            mae_result,
                                                                            r2_result*100))

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 14, 14)]     0           []                               
                                                                                                  
 layer_normalization_8 (LayerNo  (None, 14, 14)      28          ['input_2[0][0]']                
 rmalization)                                                                                     
                                                                                                  
 multi_head_attention_4 (MultiH  (None, 14, 14)      60430       ['layer_normalization_8[0][0]',  
 eadAttention)                                                    'layer_normalization_8[0][0]']  
                                                                                            

 ambda)                                                           'tf.__operators__.add_13[0][0]']
                                                                                                  
 layer_normalization_15 (LayerN  (None, 14, 14)      28          ['tf.__operators__.add_14[0][0]']
 ormalization)                                                                                    
                                                                                                  
 conv1d_14 (Conv1D)             (None, 14, 4)        60          ['layer_normalization_15[0][0]'] 
                                                                                                  
 dropout_16 (Dropout)           (None, 14, 4)        0           ['conv1d_14[0][0]']              
                                                                                                  
 conv1d_15 (Conv1D)             (None, 14, 14)       70          ['dropout_16[0][0]']             
          

<div class="alert alert-block alert-info"> <b><h3>Note:</h3></b> </div>

We ran the transformer model on google colab with GPU allocation , and were able to successfully train the model on the dataset for 12-14 hours. However, before we could save the model the colab kernel terminated and all the runtime data  got deleted. We tried running the model several time after, after it kept crashing after 7-8 epochs.
The model ran had the following test metrics score:

Rmse : 8.72
Mse: 76.352
Mae: 6.625
R2_score : 0.907



<div class="alert alert-block alert-info"> <b><h3>Conclusion:</h3></b> </div>
    
Transformer has a potential to perform better when trained for larger epochs with GPU allocation so that it takes less time to train.

<div class="alert alert-block alert-info"> <b><h3>References</h3></b> </div>



https://srdas.github.io/DLBook2/
https://github.com/mounalab/Multivariate-time-series-forecasting-keras
https://towardsdatascience.com/how-to-make-a-pytorch-transformer-for-time-series-forecasting-69e073d4061e
https://arxiv.org/abs/2205.13504
